In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils.preprocess import *
from utils.model import *

### Reduced ts

In [3]:
produv_train = pd.read_csv(config['data_path'] + 'produv_train.csv')
gas_train = pd.read_csv(config['data_path'] + 'gas_train.csv')
chronom_train = pd.read_csv(config['data_path'] + 'chronom_train.csv')

produv_test = pd.read_csv(config['data_path'] + 'produv_test.csv')
gas_test = pd.read_csv(config['data_path'] + 'gas_test.csv')
chronom_test = pd.read_csv(config['data_path'] + 'chronom_test.csv')

In [4]:
reduce_ts(ts_preproc(gas_train, 'Time'), chronom_train).to_csv(config['data_path'] + 'reduced_gas_train.csv')
reduce_ts(ts_preproc(produv_train, 'SEC'), chronom_train).to_csv(config['data_path'] + 'reduced_produv_train.csv')
reduce_ts(ts_preproc(gas_test, 'Time'), chronom_test).to_csv(config['data_path'] + 'reduced_gas_test.csv')
reduce_ts(ts_preproc(produv_test, 'SEC'), chronom_test).to_csv(config['data_path'] + 'reduced_produv_test.csv')

### Extract ts features

In [11]:
train_data = read_data('train')

In [12]:
X_train = preprocess_data(train_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2063/2063 [01:36<00:00, 21.33it/s]


In [13]:
test_data = read_data('test')
for file in files:
    test_data[file] = test_data[file][test_data[file].NPLV != 512905]

In [14]:
X_test = preprocess_data(test_data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 779/779 [00:31<00:00, 24.39it/s]


In [56]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

X_train = X_train.select_dtypes(include=numerics)
X_test  = X_test.select_dtypes(include=numerics)

In [10]:
#X_train.to_csv(config['data_path'] + 'num_extracted_features_train.csv')
# X_test.to_csv(config['data_path'] + 'num_extracted_features_test.csv')
# X_train = pd.read_csv(config['data_path'] + 'num_extracted_features_train.csv')
# X_test  = pd.read_csv(config['data_path'] + 'num_extracted_features_test.csv')

In [60]:
train_NPLV = X_train['NPLV']
test_NPLV = X_test['NPLV']
selected_features_train, selected_features_test = ts_select_features(X_train.drop(['NPLV'], axis = 1), X_test.drop(['NPLV'], axis = 1))
# selected_features_train, selected_features_test = X_train.drop(['NPLV'], axis = 1), X_test.drop(['NPLV'], axis = 1)

In [61]:
# selected_features_test = selected_features_test.drop(['Unnamed: 0'], axis = 1)
selected_features_test['NPLV'] = test_NPLV

In [62]:
imposter = pd.DataFrame(selected_features_test.mean(),columns=[779]).T
imposter['NPLV'] = [512905]

In [63]:
selected_features_test = pd.concat([selected_features_test, imposter]).sort_values('NPLV').reset_index().drop(['index'],axis=1)

In [64]:
with open('clustering_test.pickle', 'rb') as f:
    clustering_test = pickle.load(f)
    
    clustering_test_df = pd.DataFrame({'NPLV' : selected_features_test['NPLV'].values, 'cluster_label' : clustering_test})
    selected_features_test = selected_features_test.merge(clustering_test_df, on='NPLV')

In [65]:
# selected_features_train = selected_features_train.drop(['Unnamed: 0'], axis = 1)
selected_features_train['NPLV'] = train_NPLV

with open('clustering_train.pickle', 'rb') as f:
    clustering_train = pickle.load(f)
    
    clustering_train_df = pd.DataFrame({'NPLV' : train_NPLV.values, 'cluster_label' : clustering_train})
    selected_features_train = selected_features_train.merge(clustering_train_df, on='NPLV')

In [66]:
selected_features_train.shape

(2063, 5402)

In [67]:
selected_features_train.to_csv(config['data_path'] + 'preprocessed_train.csv')
selected_features_test.to_csv(config['data_path'] + 'preprocessed_test.csv')